In [55]:
# basically following this: https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html
# and https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

import gensim 
import pandas
from gensim.test.utils import datapath
from gensim.models.fasttext import load_facebook_model

MAX_SEQUENCE_LENGTH = 70

#cap_path = "./cc.en.300.bin"
#fb_model = load_facebook_model(cap_path)
#FILENAME = './hate-speech-and-offensive-language/data/labeled_data.csv'



In [16]:
from gensim.test.utils import datapath
from gensim.models.fasttext import load_facebook_model
#cap_path = "./cc.en.300.bin"
#fb_model = load_facebook_model(cap_path)


In [17]:
import numpy as np

def modified_trim_rule(word, count, min_count):
    return gensim.utils.RULE_KEEP

'assfart' in fb_model.wv.vocab
new_sentences = [['you', 'assfart']]
fb_model.build_vocab(new_sentences, trim_rule=modified_trim_rule, update=True)
fb_model.train(new_sentences, total_examples=len(new_sentences), epochs=fb_model.epochs)

print('assfart' in fb_model.wv.vocab)  # New word is now in the vocabulary
# print(fb_model.wv.most_similar(positive=['assfart'],topn=10))

NameError: name 'fb_model' is not defined

In [23]:
# Execute this prior to running test
import nltk
nltk.download('punkt')
nltk.download('stopwords')
 

[nltk_data] Downloading package punkt to /Users/drew/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/drew/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [103]:
"""
Functions for model
"""
import pandas
import gensim
import re
from gensim.test.utils import datapath
from gensim.models.fasttext import load_facebook_model
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import numpy as np


# for the four data sets 
# loop 0 to 35 and if in the sentence, replace it with the word's value from wordtovec
# if no word then put in 0 instead
TRAINING_OFFENSIVE_FILENAME = "./training_offensive.csv"
TRAINING_REGULAR_FILENAME = "./training_regular.csv"
TEST_OFFENSIVE_FILENAME = "./test_offensive.csv"
TEST_REGULAR_FILENAME = "./test_regular.csv"
LOCAL_TRAINING_OFFENSIVE_FILENAME = "./training_offensive_local.csv"
LOCAL_TRAINING_REGULAR_FILENAME = "./training_regular_local.csv"
LOCAL_TEST_OFFENSIVE_FILENAME = "./test_offensive_local.csv"
LOCAL_TEST_REGULAR_FILENAME = "./test_regular_local.csv"

#FB_MODEL_FILENAME = "./cc.en.300.bin"
#with open(LOCAL_TEST_REGULAR_FILENAME) as f:
#    print(f.read())
# fb_model = load_facebook_model(FB_MODEL_FILENAME) # do this in real code
# off_train_dataframe = pandas.read_csv(TRAINING_OFFENSIVE_FILENAME)
# off_test_dataframe = pandas.read_csv(TEST_OFFENSIVE_FILENAME)
# reg_train_dataframe = pandas.read_csv(TRAINING_REGULAR_FILENAME)
# reg_test_dataframe = pandas.read_csv(TEST_REGULAR_FILENAME)
off_train_dataframe = pandas.read_csv(LOCAL_TRAINING_OFFENSIVE_FILENAME)
off_test_dataframe = pandas.read_csv(LOCAL_TEST_OFFENSIVE_FILENAME)
reg_train_dataframe = pandas.read_csv(LOCAL_TRAINING_REGULAR_FILENAME)
reg_test_dataframe = pandas.read_csv(LOCAL_TEST_REGULAR_FILENAME)

In [25]:
stop_words = set(stopwords.words('english')) 
non_alphabet_pattern = re.compile("[^a-zA-Z]")
#NULL_VECTOR = fb_model.wv[""]

def get_cleaned_tokens(sentence):    
    word_tokens = word_tokenize(sentence) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence = [re.sub(non_alphabet_pattern, "", w) for w in filtered_sentence]
    filtered_sentence = [w for w in filtered_sentence if w != ""]
    return filtered_sentence


In [91]:
all_train_tweets = off_train_dataframe['tweet']
all_train_tweets = all_train_tweets.append(reg_train_dataframe['tweet'])

all_test_tweets = off_train_dataframe['tweet']
all_test_tweets = all_test_tweets.append(reg_test_dataframe['tweet'])

In [70]:
# tokenize words 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
VOCAB_SIZE = 100000
tokenizer = Tokenizer(nb_words=VOCAB_SIZE)
tokenizer.fit_on_texts(all_train_tweets)
sequences_train = tokenizer.texts_to_sequences(all_train_tweets)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 356 unique tokens.


In [94]:
sequences_test = tokenizer.texts_to_sequences(all_test_tweets)

In [58]:
# pad sequences
train_data = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH)
test_data = pad_sequences(sequences_test, maxlen = MAX_SEQUENCE_LENGTH)

In [92]:
all_train_labels = off_train_dataframe['offensive']
all_train_labels = all_tweet_labels.append(reg_train_dataframe['offensive'])

all_test_labels = off_test_dataframe['offensive']
all_test_labels = all_tweet_labels.append(reg_test_dataframe['offensive'])


In [101]:
print(len(all_train_tweets))
print(len(all_test_tweets))

60
60


In [66]:
embeddings_index = {}
import os
f = open(os.path.join('glove.twitter.27B.50d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1193514 word vectors.


In [79]:
from numpy import array
from numpy import asarray
from numpy import zeros
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = zeros((VOCAB_SIZE, 50)) #DIMENSIONS!
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector
    

Loaded 1193514 word vectors.


In [104]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GlobalMaxPooling1D
# define model
model = Sequential()
e = Embedding(VOCAB_SIZE, 100, weights=[embedding_matrix], input_length=70, trainable=False)
model.add(e)
model.add(Flatten()) # must use Flatter or GlobalMax
#model.add(GlobalMaxPooling1D())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#history = model.fit(offensive_vectors, non_offensive_vectors,
##                    epochs=10,
  #                  verbose=False,
   #                 validation_data=(offensive_vectors_test, non_offensive_vectors_test),
    #                batch_size=10)
#loss, accuracy = model.evaluate(offensive_vectors, non_offensive_vectors, verbose=False)
#print("Training Accuracy: {:.4f}".format(accuracy))
#loss, accuracy = model.evaluate(offensive_vectors_test, non_offensive_vectors_test, verbose=False)
#print("Testing Accuracy:  {:.4f}".format(accuracy))
#plot_history(history)

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(train_data, all_train_labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(train_data, all_train_labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 70, 100)           10000000  
_________________________________________________________________
flatten_8 (Flatten)          (None, 7000)              0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                70010     
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 11        
Total params: 10,070,021
Trainable params: 70,021
Non-trainable params: 10,000,000
_________________________________________________________________
None


ValueError: Input arrays should have the same number of samples as target arrays. Found 60 input samples and 70 target samples.

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Conv1D(filter_num, kernel_size, activation = 'relu'))
model.add(keras.layers.GlobalMaxPooling1D())
model.add(keras.layers.Dense(10, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit(offensive_vectors, non_offensive_vectors,
                    epochs=10,
                    verbose=False,
                    validation_data=(offensive_vectors_test, non_offensive_vectors_test),
                    batch_size=10)
loss, accuracy = model.evaluate(offensive_vectors, non_offensive_vectors, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(offensive_vectors_test, non_offensive_vectors_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_history(history)

In [8]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)
# load the whole embedding into memory
embeddings_index = dict()
f = open('../glove_data/glove.6B/glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()

Using TensorFlow backend.


[[6, 2], [3, 1], [7, 4], [8, 1], [9], [10], [5, 4], [11, 3], [5, 1], [12, 13, 2, 14]]
[[ 6  2  0  0]
 [ 3  1  0  0]
 [ 7  4  0  0]
 [ 8  1  0  0]
 [ 9  0  0  0]
 [10  0  0  0]
 [ 5  4  0  0]
 [11  3  0  0]
 [ 5  1  0  0]
 [12 13  2 14]]


FileNotFoundError: [Errno 2] No such file or directory: '../glove_data/glove.6B/glove.6B.100d.txt'